In [5]:
!pip install opencv-python ultralytics matplotlib mediapipe
import os
import cv2
from ultralytics import YOLO
import matplotlib.pyplot as plt

def mount_google_drive():
    """Mounts Google Drive in Colab environment."""
    from google.colab import drive
    drive.mount('/content/drive')

model = YOLO("yolo12n.pt")

In [7]:
import mediapipe as mp
import numpy as np

# Ask user for height in cm
user_height_cm = float(input("Enter your height in cm (e.g., 175): "))

# Initialize Mediapipe Pose
mp_pose = mp.solutions.pose
pose = mp_pose.Pose(static_image_mode=False, model_complexity=2)

# Initialize video processing
cap = cv2.VideoCapture('/content/dresscombo.mp4')
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = cap.get(cv2.CAP_PROP_FPS)

# Output writer setup
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter('/content/output_dressed.mp4', fourcc, fps, (width, height))

def estimate_size_and_weight(landmarks, height_cm):
    # Key points
    shoulder_y = landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y
    hip_y = landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].y
    knee_y = landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].y
    ankle_y = landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].y

    # Convert to actual measurements
    torso_len = abs(hip_y - shoulder_y)
    leg_len = abs(ankle_y - hip_y)
    full_body_len = abs(ankle_y - shoulder_y)

    scaling_factor = height_cm / full_body_len

    # Rough approximations
    trouser_length_cm = leg_len * scaling_factor
    weight_kg = (height_cm - 100) + np.random.uniform(-5, 5)  # Broca index with small noise
    shirt_size = "M" if height_cm < 165 else "L" if height_cm < 180 else "XL"
    trouser_size = "30" if weight_kg < 65 else "32" if weight_kg < 75 else "34"

    return shirt_size, round(weight_kg), round(trouser_length_cm), trouser_size

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # Mediapipe expects RGB
    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results = pose.process(rgb_frame)

    if results.pose_landmarks:
        lms = results.pose_landmarks.landmark
        shirt_size, weight, trouser_len, trouser_size = estimate_size_and_weight(lms, user_height_cm)

        # Draw annotations
        mp.solutions.drawing_utils.draw_landmarks(frame, results.pose_landmarks, mp_pose.POSE_CONNECTIONS)

        # Overlay info
        cv2.putText(frame, f"Shirt: {shirt_size}, Trouser: {trouser_size}", (10, 40), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255, 0, 0), 2)
        cv2.putText(frame, f"Est. Weight: {weight} kg, Trouser Len: {trouser_len} cm", (10, 70), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255, 0), 2)

    out.write(frame)

cap.release()
out.release()
pose.close()
print("✅ Output video saved at /content/output_dressed.mp4")

# Final average estimation based on height alone
predicted_weight = round((user_height_cm - 100) * 0.9)  # Broca Index approximation
shirt_size_final = "M" if 38 <= predicted_weight <= 70 else ("S" if predicted_weight < 38 else "L")
trouser_length_final = round(0.45 * user_height_cm)
trouser_size_final = "30" if predicted_weight < 55 else ("32" if predicted_weight <= 70 else "34")

print("\n----- Estimated Details Based on Height -----")
print(f"Estimated Weight: {predicted_weight} kg")
print(f"Shirt Size: {shirt_size_final}")
print(f"Trouser Size: {trouser_size_final}")
print(f"Trouser Length: {trouser_length_final} cm")

Enter your height in cm (e.g., 175): 170
✅ Output video saved at /content/output_dressed.mp4

----- Estimated Details Based on Height -----
Estimated Weight: 63 kg
Shirt Size: M
Trouser Size: 32
Trouser Length: 76 cm
